## **Code for Blunt Malacology's Phylogenetic Pipeline**

| Standard csv file formats  | | | | |
|-----------|---------|---------|-|-|
| **Ascension file** | Taxon | Gene1 | Gene2 | ... |
| **Enum file** | Enum_id | Taxon |||
| **Description file** | Enum_id | Taxon | Ascension_number | Taxon_description |
| **Gene length** | Gene | Length | | |

**Standard fasta format for genes:**

>\><enum_id> Genus_species<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ACTG...


#### Logging functions
 * log(string) - logs a given string
 * checkpoint(head,count,total=0,remove=True) - logs a string in the form "    \<head\>: checkpoint \<count\>/\<total\>" which can be flushed. Returns the count number + 1

In [19]:
""" LOGS A GIVEN STRING """
def log(inp):
    print(inp)

In [2]:
""" Forms a checkpoint
Prints the checkpoint position out of a total number
*TODO MAY CHANGE TO PERCENTAGE*
TODO Clears the string buffer beforehand

returns the next checkpoint number
"""
def checkpoint(head,count,tot = 0,remove=True):
    to_log = ""
    to_log += "\t" + head + ": checkpoint " + str(count)
    if (tot != 0):
        to_log += "/" + str(tot)
    log(to_log)

    return count+1

#### Util functions
 * create_enum (input_filename) - creates an enum file from an input ascension file
 * read_enum (enum_filename) - reads an enum file, returning the dictionary
 * get_fasta_filenames (directory) - returns the fastas in the given directory

In [3]:
import csv

"""
Create enumerated file

/param ascension_filename - a string of the csv input_ascensions file to open & create the enumerated file for
    in the form: | Taxon | Gene1 | Gene2 ...

Creates an enum_<input_filename>, should end with .csv
    in the form | Enum_id | Taxon |
                |   int   |  str  |
"""
def create_enum(ascension_filename):
    log("Creating enumeration file for " + ascension_filename)

    ascension_file = open(ascension_filename, 'r')
    ascension_reader = csv.DictReader(ascension_file)

    # Setting the first column to be right - often excel changes this
    ascension_fieldnames = ascension_reader.fieldnames
    ascension_fieldnames[0] = "Taxon"

    enum_filename = "enum_" + ascension_filename
    enum_file = open(enum_filename, 'w')
    enum_fieldnames = ["Enum_id","Taxon"]
    enum_writer = csv.DictWriter(enum_file,fieldnames=enum_fieldnames)

    enum_writer.writeheader()

    enum = 0
    for ascension_row in ascension_reader:
        enum_row = {enum_fieldnames[0]:enum, enum_fieldnames[1]:ascension_row["Taxon"]}
        enum_writer.writerow(enum_row)
        enum+=1
    
    ascension_file.close()
    enum_file.close()

    return enum_filename

#log(create_enum("input_ascension.csv"))

In [ ]:
import csv
"""
Example use of enumeration above
"""

####                                   ####
"""
Read enumerated file

/param enum_filename - a path ot the csv input file
    in the form | Enum_id | Taxon |
                |   int   |  str  |

/return a dictionary with a csv file with an enum number column & a taxon name column
"""
#### LEAVES FILE OPEN, MUSTN'T BE USED ####

"""
enum_filename = create_enum("input_ascension.csv")

enum_file = open(enum_filename, 'r')
enum_reader = csv.DictReader(enum_file)

for row in enum_reader:
    log(str(row["Enum_id"]) + ": " + row["Taxon"])

enum_file.close()
"""

In [20]:
import os

"""
A function that gets the names of all the fasta files in a given directory
"""
def get_fasta_filenames(directory):
    # Get all files in the directory
    files = os.listdir(directory)

    # Filter only fasta files
    return [file for file in files if file.endswith('.fasta')]


#### Get fasta from ascension
 * A function to get a fasta for a given gene & a csv for their descriptions
 * A loop running through these

In [5]:
from Bio import Entrez
from Bio import SeqIO


"""!
Takes a set of ascension numbers for the same gene, and creates an unaligned fasta of them

/param filename
    the name of the output file to be created
/param CSVfile
    Already loaded into a csv.DictReader. In the format
        taxon name , gene 1 , gene 2 , etc
        .
        .
        .
    Where the genes are ascension numbers
/param gene
    the header name for the gene to create a fasta for
/param EntrezEmail
    an email address for the NCBI, so the US government can track

The output is a set of fastas as outputted from the Bio SeqIO with the headers replaced by the taxon names
& a file in which the original headers are situated
 - called filename_data.fasta

As well as a descriptions.csv file
"""

def fastaFromAscensions(filename,CSVfile,CSVfile_linenum,gene,EntrezEmail):
    Entrez.email = EntrezEmail # Telling NCBI who I am

    # Setting up checkpoints
    chk_count = 0
    chk_total = CSVfile_linenum + 3 # + 2 for the checkpoints before going into the loop, + 1 for the one afterwards
    chk_count = checkpoint(gene,chk_count,chk_total)

    # The genes
    listify = []
    # The descriptions
    dscrptsname = gene + "_descriptions.csv"
    descptsfile = open(dscrptsname, 'w')
    dscrptswriter = csv.writer(descptsfile)
    dscrpts = []
    enum_id = 0

    chk_count = checkpoint(gene,chk_count,chk_total)

    for row in CSVfile:

        chk_count = checkpoint(gene,chk_count,chk_total)

        if (row[gene]!='' and row[gene]!=gene):
            # Fetching the fasta from the nucleotide site, then putting it as a SeqRecord type
            handle = Entrez.efetch(db="nucleotide", id=row[gene], rettype="fasta", retmode="text")
            record_in = SeqIO.read(handle,"fasta") # Parses it, then chooses the right one
            handle.close()

            taxon_name = row["Taxon"].replace(" ","_")

            # Recording the descriptions
            dscrpts = [str(enum_id),taxon_name,record_in.id,record_in.description]
            dscrptswriter.writerow(dscrpts)

            # Creating a clean record with an enumerated id & user-defined description
            record_out = SeqIO.SeqRecord(record_in.seq,id = str(enum_id), description = taxon_name)
            listify.append(record_out)
            
        enum_id += 1;
    

    # Writing the .fasta file
    SeqIO.write(listify,filename,"fasta")

    # Cleaning up
    descptsfile.close()
    chk_count = checkpoint(gene,chk_count,chk_total)


In [ ]:
import csv
"""
Just doing one gene - if needed
"""

### GETTING INFORMATION BEFORE THE EXTRACTION ###
email = "rhys.edmunds@yahoo.com" # TODO will be in the settings.csv

# Opening the file into a usable format
ascension_filename = "input_ascension.csv"
file = open(ascension_filename, mode="r")
CSVfile = csv.DictReader(file) # DictReader means that the first row is used as monikers for the columns

# Creating the enum file
enum_filename = create_enum(ascension_filename)

# Formatting the csv file's 
gene_list = CSVfile.fieldnames.copy()
if gene_list[0] != "Taxon":
    gene_list[0] = "Taxon"

# Getting the gene names
fieldnames = gene_list.copy()

# Getting only the genes
gene_list.remove("Taxon")
log("GENE LIST: " + str(gene_list))

# Getting the length of the file
enum_file = open(enum_filename, 'r')
file_length = len(enum_file.readlines()) - 1 # For the header
enum_file.close()

### RUNNING THE PROCESS ###
# Processing the gene
gene = "12S"
log(gene + " [PROCESSING]")
filename = str(gene) + ".fasta"

# Setting the CSVfile fieldnames
CSVfile.fieldnames = fieldnames

# TODO TEST WITH .readlines(), make sure it doesn't iterate through
fastaFromAscensions(filename,CSVfile,file_length,gene,email)

# Closing the file
file.close()



In [ ]:
import csv
"""
A loop going through the above function for each gene

Input - input ascension described above
Output - A set of gene files & a .csv for the descriptions
"""


email = "rhys.edmunds@yahoo.com" # TODO will be in the settings.csv

# Opening the file into a usable format
ascension_filename = "input_ascension.csv"
file = open(ascension_filename, mode="r")
CSVfile = csv.DictReader(file) # DictReader means that the first row is used as monikers for the columns

# Creating the enum file
create_enum(ascension_filename)

# Formatting the csv file's 
gene_list = CSVfile.fieldnames.copy()
if gene_list[0] != "Taxon":
    gene_list[0] = "Taxon"

# Getting the gene names
fieldnames = gene_list.copy()

# Getting only the genes
gene_list.remove("Taxon")
log("GENE LIST: " + str(gene_list))

# Getting the length of the file
enum_file = open(enum_filename, 'r')
file_length = len(enum_file.readlines())
enum_file.close()

# Writing the four separate .fasta files in the form
"""
><ascension> <taxon>
<sequence>
.
.
"""
for gene in gene_list:
    # Opening a file & setting the CSV DictReader if the file isn't open already
    if file.closed:
        file = open("input_ascension.csv", mode="r")
        CSVfile = csv.DictReader(file)
    
    # Setting the CSVfile fieldnames
    CSVfile.fieldnames = fieldnames

    # Processing the gene
    log(gene + " [PROCESSING]")
    filename = str(gene) + ".fasta"

    # TODO TEST WITH .readlines(), make sure it doesn't iterate through
    fastaFromAscensions(filename,CSVfile,len(file.readlines()),gene,email)

    # Closing the file
    file.close()


#### Concatenate fastas

In [107]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Seq import MutableSeq

"""
A function that concatenates two genes
Genes should have an enumerated id, 0-number of taxa at the beginning
& a taxon name
e.g.
    >12 Siratus_pliciferoides
        TCGAGA...

CURRENTLY RELIES ON 0 HAVING ALL GENES TODO

/param ins - a list of filenames for the relevant fastas
/param enum_filename - a filename for the enumerated file for the genes
/param out - name of the fasta file to concatenate to, WITHOUT fasta ending

writes a fasta file with the enumerated ids, the taxon names & the concatenated genes named
writes a gene length file for the genes
"""
def concatenateFastas(ins,enum_filename,out):
    # Setting up checkpoints
    chk_count = 0
    chk_total = 3
    
    chk_count = checkpoint(out,chk_count,chk_total)
    
    # Getting the enum file
    enum_file = open(enum_filename, 'r')
    enum_reader = csv.DictReader(enum_file)

    # List to write out the sequences
    out_list = []

    # File that writes the gene length file
    gene_len_filename = out[:-6] + "_gene_lengths.csv" # filename without the fasta
    gene_len_file = open(gene_len_filename,'w')
    gene_len_fieldnames = ["Gene","Length"]
    gene_len_writer = csv.DictWriter(gene_len_file,fieldnames=gene_len_fieldnames)
    # For iterating through the genes
    gene_num = 0

    chk_count = checkpoint(out,chk_count,chk_total)
    
    for enum_row in enum_reader:
        # Creating a sequence & appending the right sequences upon them - the first of a given record only

        # Variables for the enum id, in integer & string forms
        enum_id = int(enum_row["Enum_id"])
        enum_id_str = str(enum_row["Enum_id"])
        # log(str_enum_id)
        
        # The sequence to be appended
        temp_sequence = MutableSeq("")

        for fasta_name in ins:
            if enum_id == 0:
                continue
            
            fasta = SeqIO.index(fasta_name,'fasta')
            # for k in fasta.keys():
            #     log(k + ": " + fasta[k].description)

            # Getting the genes and their lengths, creating the gene length file
            if enum_id == 1:
                gene_len_row = {gene_len_fieldnames[0]:fasta_name[:-6],gene_len_fieldnames[1]:len(fasta[enum_id_str].seq)}
                gene_num += 1
                gene_len_writer.writerow(gene_len_row)
            
            if enum_id_str in fasta.keys():
                # Appending the sequence
                temp_sequence = temp_sequence + fasta[enum_id_str].seq
            # Adding a None sequence to maintain the correct length if the append wasn't the correct length
            else:
                zero_sequence = "-" * len(fasta['1'].seq)
                temp_sequence = temp_sequence + zero_sequence # ASSUMING fasta[0] HAS THE GENE
            
            fasta.close()
            
            # Adding to the list for the file to be put out
            if (fasta_name == ins[-1]):
                out_list.append(SeqRecord(temp_sequence,enum_row["Taxon"]))

        #out_list.append(SeqRecord(temp_sequence,enum_row["Taxon"]))

    chk_count = checkpoint(out,chk_count,chk_total)
    
    # Writing the file
    SeqIO.write(out_list, (out + '.aln'), 'fasta')

    # Closing all the files
    gene_len_file.close()
    enum_file.close()

    chk_count = checkpoint(out,chk_count,chk_total)
    


In [108]:

"""
Runs the above code using all the fastas in the current folder as the input
"""

fasta_list = get_fasta_filenames('.') # Eventually, this will be set in the folder settings
all_fastas = ""
for f in fasta_list:
    all_fastas += f[:-6]

filename = "concatenated" + all_fastas
log("Concatenating to " + filename + ".fasta:")

enum_filename = "enum_input_ascension.csv"

concatenateFastas(fasta_list,enum_filename,filename)

Concatenating to concatenated12S16S28SCOI:
	concatenated12S16S28SCOI: checkpoint 0/3
	concatenated12S16S28SCOI: checkpoint 1/3
	concatenated12S16S28SCOI: checkpoint 2/3
	concatenated12S16S28SCOI: checkpoint 3/3


In [ ]:
"""
Testing for developing the above

"""
"""
to_append = Seq("CAGTC")
zero_sequence = "-" * 4
temp_sequence = MutableSeq("")
temp_sequence = temp_sequence + to_append + zero_sequence
#temp_sequence = temp_sequence + zero_sequence
log(temp_sequence)
"""

#### Translate & count stop codons

Based on from Bio.Data import CodonTable,
Translates protein coding & counts the number of stop codons

#### Fasta to Nexus

In [22]:
from Bio import SeqIO

IUPAC_ambiguous = 'MRWSYKVHDBN' # Global variables

"""
Removes IUPAC ambigous characters from a record

/param ambig_filename - a fasta
/param ambigs - a (const) list of characters that represent the ambiguous characters

/return - the same records but with the ambiguous characters replaces by '_'
"""
def remove_ambiguous (ambig_filename, ambig_chars = IUPAC_ambiguous, replace_char = '-'):
    ambig_file = open(ambig_filename,'r')
    unambig_file = open("unambiguous_"+ambig_filename,'w')
    for ambig_line in ambig_file.readlines():
        # Missing the headers
        if (ambig_line[0] == '>'):
            unambig_file.write(ambig_line)
            continue
        
        new_line = ambig_line
        for ambig_char in ambig_chars:
            new_line = new_line.replace(ambig_char,replace_char)

        unambig_file.write(new_line)
    
    return "unambiguous_"+ambig_filename

#print(remove_ambiguous('12S.fasta'))

In [47]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

"""
Function that changes fasta to nexus

/param - input_handle: an open file for the fasta that's being read WITHOUT '.fasta'
/param - filename: a filename for the given file to be created
- Writes a nexus file
/return - nexus file of the input file
"""
def fasta_to_nexus (fasta_filename, nexus_filename,ambiguous=True):
    with open(fasta_filename + '.fasta', "rU") as input_handle:
        with open(nexus_filename+".nexus", "w") as output_handle:
            records = SeqIO.parse(input_handle, "fasta")
            records_out = []
            for record in records:
                if (record.description[-1]=='>'): # Removing ' <unknown description>' occurring on concatenated
                    record.description = record.description[:-22]
                records_out.append(SeqRecord(seq=record.seq,id=record.description.replace(' ','_'),annotations={"molecule_type": "DNA"}))
            
            SeqIO.write(records_out, output_handle, "nexus")


In [48]:
# Running the above functions
gene_list = get_fasta_filenames(".")

# Removing the file endings ".fasta", the last 6 characters
gene_list = [g[:-6] for g in gene_list]

for gene in gene_list:
    fasta_to_nexus(gene,gene)


#### Creating empty

In [4]:
no_28S = 1425
no_COI = 656
no_16S = 581
no_12S = 447

print("-" * no_12S)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
